<a href="https://colab.research.google.com/github/merttezcan/fakenews-twitter/blob/main/fakenews_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

In [ ]:
import pandas as pd
import numpy as np
import dgl
import torch as th

In [ ]:
#loading the nodes and features, replacing ids with new consecutive ids starting from zero.
#also doing some preprocessing

nodes_data = pd.read_csv('gilani-2017-features.csv')
nodes_data.rename(columns={'id': 'tw_id'}, inplace=True)
nodes_data.insert(0, 'new_id', range(0, 0 + len(nodes_data)))
nodes_data.drop(['name','screen_name','url','lang','time_zone','location','default_profile',
                           'default_profile_image','geo_enabled','profile_image_url','profile_use_background_image',
                           'profile_background_image_url_https','profile_text_color','profile_image_url_https',
                           'profile_sidebar_border_color','profile_background_tile','profile_sidebar_fill_color',
                           'profile_background_image_url','profile_background_color','profile_link_color','utc_offset',
                           'description','created_at'], axis=1, inplace=True)

nodes_data.replace({'class': {'human': True, 'bot': False}}, inplace=True)

display(nodes_data)

In [ ]:
#creating a small df to have tw_id and new_id, then converting it to a dictionary

df= nodes_data[['tw_id', 'new_id']]
df = df.astype('int64')
id_dict = dict(df.values)

In [ ]:
#checking our dict

print(id_dict)

In [ ]:
#loading the friends file that contains user id and friend id

edges_data = pd.read_csv("friends.csv",names=['user_id', 'friend_id'])
edges_data.dropna(inplace=True)

edges_data = edges_data.astype('int64')

print(edges_data)

In [ ]:
#using the dict that we created before, matching the ids with the new ones

edges_data['fr_source'] = edges_data['user_id'].map(id_dict)
edges_data['fr_target'] = edges_data['friend_id'].map(id_dict)


display(edges_data)

In [ ]:
#dropping the null values, that means we are removing the users that are not connected to each other

fr_edges_df = edges_data.dropna()
fr_edges_df = fr_edges_df.astype('int64')

In [ ]:
display(fr_edges_df)

In [ ]:
#doing same process for the followers file

edges_data2 = pd.read_csv("followers.csv",names=['user_id', 'follower_id'])
edges_data2.dropna(inplace=True)

edges_data2 = edges_data2.astype('int64')

print(edges_data2)

In [ ]:
edges_data2['fw_source'] = edges_data2['user_id'].map(id_dict)
edges_data2['fw_target'] = edges_data2['follower_id'].map(id_dict)


display(edges_data2)

In [ ]:
fw_edges_df = edges_data2.dropna()
fw_edges_df = fw_edges_df.astype('int64')

In [ ]:
display(fw_edges_df)

In [ ]:
# Create an empty list
fr_list =[]

# Iterate over each row
for rows in fr_edges_df.itertuples():
	# Create list for the current row
	my_list =[rows.fr_source, rows.fr_target]
	
	# append the list to the final list
	fr_list.append(my_list)

# Print the list
print(fr_list)

In [ ]:
# Create an empty list
fw_list =[]

# Iterate over each row
for rows in fw_edges_df.itertuples():
	# Create list for the current row
	my_list =[rows.fw_source, rows.fw_target]
	
	# append the list to the final list
	fw_list.append(my_list)

# Print the list
print(fw_list)

In [ ]:
#creating an heterograph with using our edge lists

g = dgl.heterograph(
    {('user', 'friend', 'user'): fr_list,
     ('user', 'follow', 'user'): fw_list
     })
g

In [ ]:
g['friend'].edges()

In [ ]:
#a function for creating tensors for features easily

def tensorCreator(df):

  i = 1

  for column in df.columns[1:]:
    globals()["t"+ str(i)] = th.tensor(df[column].values)
    i += 1

tensorCreator(nodes_data)   

In [ ]:
#adding the tensors to the graph

g.ndata['class'] = t1
g.ndata['tw_id'] = t2
g.ndata['statuses_count'] = t3
g.ndata['followers_count'] = t4
g.ndata['friends_count'] = t5
g.ndata['favourites_count'] = t6
g.ndata['listed_count'] = t7
g.ndata['is_translator'] = t8
g.ndata['follow_request_sent'] = t9
g.ndata['protected'] = t10
g.ndata['verified'] = t11
g.ndata['notifications'] = t12
g.ndata['contributors_enabled'] = t13
g.ndata['following'] = t14

In [ ]:
#done! we can check all of the nodes' features. for example:
g.nodes[2400]

In [ ]:
!pip install node2vec

In [ ]:
import networkx as nx
from node2vec import Node2Vec

In [ ]:
KGF = nx.Graph(fr_list)

In [ ]:
KGF.number_of_nodes(), KGF.number_of_edges()

In [ ]:
KGFol = nx.Graph(fw_list)

In [ ]:
KGFol.number_of_nodes(), KGFol.number_of_edges()

In [ ]:
node2vec = Node2Vec(KGF, dimensions=32, walk_length=30, num_walks=200, workers=4)

In [ ]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
node2vec_fw = Node2Vec(KGFol, dimensions=32, walk_length=30, num_walks=200, workers=4)

In [ ]:
model_fw = node2vec_fw.fit(window=10, min_count=1, batch_words=4)

In [ ]:
arr = []
arr2 = []
for i in range(0, KGF.number_of_nodes()):
  arr.append(model.wv.vectors[i])
  arr2.append(i)

In [ ]:
n_dict = dict(zip(arr2, arr))

In [ ]:
print(len(n_dict))

In [ ]:
#check
print(model.wv.vectors[7])
print(n_dict[7])

In [ ]:
fw_arr = []
fw_arr2 = []
for i in range(0, KGFol.number_of_nodes()):
    fw_arr.append(model_fw.wv.vectors[i])
    fw_arr2.append(i)

In [ ]:
n_dict_fw = dict(zip(fw_arr2, fw_arr))

In [ ]:
#check
print(model_fw.wv.vectors[500])
print(n_dict_fw[500])

In [ ]:
embedding_df_fr = pd.DataFrame.from_dict(n_dict, orient='index').reset_index()

In [ ]:
col_list = []
for i in range (0,32):
    col_list.append("f" + str(i))
    
col_list.insert(0, "id")

embedding_df_fr.columns = col_list

In [ ]:
list2 = col_list.copy()

In [ ]:
list2.remove('id')

In [ ]:
display(embedding_df_fr)

In [ ]:
embedding_df_fw = pd.DataFrame.from_dict(n_dict_fw, orient='index').reset_index()

In [ ]:
embedding_df_fw.columns = col_list

In [ ]:
display(embedding_df_fw)

In [ ]:
embedding_df_fr.rename(columns={'id': 'new_id'}, inplace=True)

In [ ]:
display(embedding_df_fr)

In [ ]:
embedding_df_fw.rename(columns={'id': 'new_id'}, inplace=True)

In [ ]:
display(embedding_df_fw)

In [ ]:
final_data = pd.concat((embedding_df_fr, embedding_df_fw))
final_data = final_data.groupby(final_data.index).mean()

In [ ]:
display(final_data)

In [ ]:
from sklearn.utils.extmath import softmax
emb_arr = final_data.drop(columns='new_id').to_numpy()

In [ ]:
print(emb_arr)

In [ ]:
s_arr = softmax(emb_arr)

In [ ]:
print(s_arr)

In [ ]:
softmax_df = pd.DataFrame(s_arr, columns=list2)
display(softmax_df)

In [ ]:
id_value = final_data['new_id']
print(id_value)

In [ ]:
softmax_df.set_index(keys=id_value, inplace=True)

In [ ]:
display(softmax_df)

In [ ]:
softmax_df['new_id'] = softmax_df.index

In [ ]:
softmax_df = softmax_df[ ['new_id'] + [ col for col in softmax_df.columns if col != 'new_id' ] ]
softmax_df.index.name = None

In [ ]:
display(softmax_df)

In [ ]:
merged_final_data = pd.merge(nodes_data, softmax_df, how='outer', on='new_id')

In [ ]:
merged_final_data = merged_final_data.dropna()

In [ ]:
display(merged_final_data)

In [ ]:
label = merged_final_data['class']
merged_final_data = merged_final_data.drop(columns='class')
merged_final_data.insert(loc=46, column='label', value=label)
merged_final_data = merged_final_data.drop(columns=['new_id', 'tw_id'])

In [ ]:
display(merged_final_data)

In [ ]:
X = merged_final_data.iloc[:,:-1]

y = merged_final_data.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
scaled_df = pd.DataFrame(X_train)
display(scaled_df)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=20, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [50, 150, 250],
    'max_features': ['auto', 'sqrt',  0.5, 1.0],
    'min_samples_split' : [2, 4, 6],
}


In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features=0.5, n_estimators= 150, min_samples_split=4)

In [ ]:
rfc1.fit(X_train, y_train)

In [ ]:
pred = rfc1.predict(X_test)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

In [ ]:
from sklearn.feature_selection import SelectFromModel
feat_sel = SelectFromModel(clf,threshold=0.013)
feat_sel.fit(X_train, y_train)

In [ ]:
X_imp_train = feat_sel.transform(X_train)
X_imp_test = feat_sel.transform(X_test)

In [ ]:
CV_rfc_imp = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc_imp.fit(X_imp_train, y_train)

In [ ]:
CV_rfc_imp.best_params_

In [ ]:
rfc_imp = RandomForestClassifier(random_state=42, max_features=0.5, n_estimators=250, min_samples_split=2)

In [ ]:
rfc_imp.fit(X_imp_train, y_train)

In [ ]:
pred_imp = rfc_imp.predict(X_imp_test)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred_imp))